## Verarbeitung des Datensatzes (Part III)


### Importieren der benötigten Bibliotheken

In [ ]:
import pandas as pd
import numpy as np

import os
import glob

# Machine Learning Bibliotheken
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

# Geofencing Bibliotheken
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

# Plotting Bibliotheken
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.style.use(['mystyle'])

### Einlesen des Datensatzes

In [ ]:
pathToMaschenFile = os.path.join("..", "..", "data", "maschenPickled.pickle")
dfMaschen = pd.read_pickle(pathToMaschenFile)

In [ ]:
dfMaschen

# Vorüberlegungen und erste Analysen des Datensatzes

### "Extract relevant features for your clustering model"
- führt zu einer Anwendung von "Feature Selection"
- auch "Dimesionality Reduction" relevant

---> Link: https://www.datacamp.com/community/tutorials/feature-selection-python

In [ ]:
# darstellen von bestimmten Variablen mit BoxPlots
fig, ax = plt.subplots(ncols=2, figsize=(7,4))
sns.boxplot(ax = ax[0], data=dfMaschen[['latitude']])
sns.boxplot(ax = ax[1], data=dfMaschen[['longitude']])
plt.show()

In [ ]:
# pandas DataFrame in numpyArray umwandeln um schneller Rechenzeiten zu erzielen
#array = dfMaschenMapping.values
#array

In [ ]:
# kopieren des DataFrames, damit urpsrünglicher DataFrame unverändert (da Referenztyp)
# und dennoch ohne Features, die direkt als unnütz angesehen wurden (werden)

# sinnvoll zu überprüfen sind die folgenden Variablen_
# provider
# wagon_construction
# wagon_type

dfRelevantFeatures = dfMaschen[[
    'loading_state',
    'latitude',
    'longitude',
    'GNSS_velocity',
    'movement_state']].copy()

In [ ]:
# Verwenden der Pearson Correlation
# um einen Überblick über die Korrelationen zwischen den verschiedenen Daten zu erlangen
plt.figure(figsize=(25,10))
correlation = dfRelevantFeatures.corr()
sns.heatmap(correlation, annot=True, cmap=plt.cm.Reds)
plt.show()

### "Develop a clustering model: you might gain a first overview with features depending on the geofence or the wagon moving state"

In [ ]:
# Überblick durch Geodaten und the wagon moving state"
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10,10))
ax.axis('equal')
ax.scatter(dfMaschen['latitude'],
           dfMaschen['longitude'],
           c = dfMaschen['movement_state'], s = 0.5)
plt.show()

In [ ]:
# Geodaten aufgeteilt nach dem "movement_state"
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(10,8), sharey=True, sharex=True)
for i, axis in enumerate(ax): 
    #axis.axis('equal') -> funktioniert nicht, wenn beide Axen geteilt werden
    axis.scatter(dfMaschen['latitude'].loc[dfMaschen['movement_state']==i],
                    dfMaschen['longitude'].loc[dfMaschen['movement_state']==i], s = 0.1)
    axis.set_xlabel('latitude')

ax[0].set_title('movement_state = parking')
ax[1].set_title('movement_state = standing')
ax[2].set_title('movement_state = moving')
ax[0].set_ylabel('longitude')

plt.show()

In [ ]:
# Verwenden eines Min-Max-Scalers
# Variablen und Indizes aus dem DataFrame
variables = dfRelevantFeatures.columns
indexes = dfRelevantFeatures.index
# Verwenden des MinMaxScalers aus sklearn --> siehe import der Bibliotheken
mmsc = MinMaxScaler((0, 1))
dataScaled = mmsc.fit_transform(dfRelevantFeatures)
dfScaled = pd.DataFrame(dataScaled, columns=variables, index=indexes)
dfScaled.head()

In [ ]:
# KMeans Clustering
kmeans = KMeans()
ssd = []
# Zwischen 1 und 30 Cluster werden getestest
K = range(1, 30)

for k in K:
    # Testweise werden nur die ersten 20000 Datenpunkte betrachtet
    kmeans = KMeans(n_clusters=k).fit(dfScaled.head(20000))
    ssd.append(kmeans.inertia_)

ssd

plt.plot(K, ssd, "bx-")
plt.xlabel("Distance Residual Sums for K Values (WCSS)")
plt.title("Elbow Method for Optimum Number of Clusters")
plt.show()

# Visualisierung mit KElbowVisualizer (Yellowbrick --> muss installiert werden)
#kmeans = KMeans()
#visu = KElbowVisualizer(kmeans, k=(2, 20))
#visu.fit(df)
#visu.show()

# Ansatz
Aus den Geodaten und der timestapmp_geo eine Richtung definieren -> Norden und Süden
-> als alternatives Feature zu timestamp

**Hoffnung:** Trennung der Wagons zwischen Nord Richtung und Süd Richtung

In [ ]:
dfMaschenSafe = dfMaschen.copy() # als BackUp

## Preproc df for better results

In [ ]:
print('Länge davor '+str(len(dfMaschen)))
dfMaschen.drop(columns='is_in_maschen', inplace=True)
dfMaschen.drop(dfMaschen[dfMaschen['determination_position']!=1].index, inplace=True)
print('Länge nach det_pos !=1 gedropped: '+str(len(dfMaschen)))
dfMaschen.drop(dfMaschen[dfMaschen['signal_quality_hdop']>2].index, inplace=True)
print('Länge nach signal_quality_hdop >2 gedropped: '+str(len(dfMaschen)))
#dfMaschen.drop(dfMaschen[(dfMaschen['GNSS_velocity']<1.2) & (dfMaschen['movement_state']==2)].index, inplace=True)
#print(print('Länge nach GNSS_velocity >2: '+str(len(dfMaschen)))


+ Löschen der Rides, indem ein GNNS velocity Wert größer als 50 km/h beträgt (25 km/h filtert immernoch nur um Maschen)

In [ ]:
Ride_tooFast = dfMaschen[(dfMaschen['GNSS_velocity']>50)].Ride.unique()
print('Anzahl an zu schnellen Fahrten: '+str(len(Ride_tooFast)))
print('Länge Df vor Bereinigung: '+str(len(dfMaschen)))
for ride in Ride_tooFast:
    dfMaschen.drop(dfMaschen[dfMaschen['Ride']==ride].index, inplace=True)
print('Länge Df nach GNSS velocity kleiner gleich 50 km/h: '+str(len(dfMaschen)))

+ Herausfiltern der Rides, die nur 10 oder weniger Einträge besitzen

In [ ]:
dfMaschen.sort_values('Ride', inplace=True)
countRides = dfMaschen.Ride.value_counts()
print('Anzahl an einzelnen Fahrten: '+str(len(countRides)))
shortRide = countRides.keys()[countRides.values<10]
print('Anzahl an Fahrten mit weniger als 10 Einträgen: '+str(len(shortRide)))

for Ride in shortRide:
    dfMaschen.drop(dfMaschen[dfMaschen.Ride == Ride].index, inplace=True)

print('Anzahl an Einträgen nach Entfernen kurzer Fahrten: '+str(len(dfMaschen)))

## Feature Engineering [3 features, DBSACN]

### Create a new Df with extra column of moving_direction 

    
|number|moving direction|Ansatz|Anwenung auf|
|:-:|:-|:-|:-|
|2|Fahrt nach Norden|Erster Lat Eintrag muss größer als Mittelwert aller Lat sein|ganzen Ride|
|1|Fahrt nach Süden|Erster Lat Eintrag muss kleiner als Mittelwert aller Lat sein|ganzen Ride|
|-1|everything else|

DataFrame um Spalte _moving_direction_ erweitert 

### Create a new Df with extra column of moving_direction2 

    
|number|moving direction|Ansatz|Anwenung auf|
|:-:|:-|:-|:-|
|2|Fahrt nach Norden|Summe der (Lat/Long) Differenz muss (größer/kleiner) als (positive Länge/ negative Breite) von Maschen entsprechen|ganzen Ride|
|1|Fahrt nach Süden|Summe der (Lat/Long) Differenz muss (kleiner/größer) als (negaitve Länge/ positive Breite) von Maschen entsprechen|ganzen Ride|
|-1|everything else|

DataFrame um Spalte _moving_direction2_ erweitert 

### Create a new Df with extra column of moving_direction3 

    
|number|moving direction|Anwenung auf|
|:-:|:-|:-|
|2|(Lat/Long) Differenz muss (positiv/negativ)|einzelne Einträge|
|1|(Lat/Long) Differenz muss (negativ/positiv)|einzelne Einträge|
|0|(Lat/Long) Differenz gleich 0|einzelne Einträge|
|-1|everything else|

-> mit der Annahme, dass die Moving_Direction vom 1. Geopunkt dem 2. Geopunkt entspricht

DataFrame um Spalte _moving_direction3_ erweitert 

### Create a new Df with extra column of movement count

|count|explanation|
|:-:|:-|
|0|driving and first parking|
|1|again driving and 2. parking|
|2|again driving and 3. parking|
|3|...|

DataFrame um Spalte _movement_count_ erweitert

Erweiterter DataFrame: **dfMaschen_md**



In [ ]:
#from collections import Counter

# damit man nur 1-mal die Anzahl an Zeilen definieren muss
df_MaschenStart = dfMaschen
# Vordefinieren, wie die Geodaten gefiltert werden sollen: ['lat and long with sensor inaccuracy' 'lat and long without sensor inaccuracy' 'only lat up and down']
GeoFilter = 'only lat up and down'

########################## also create a List with all single rides in it
list_splitRide = {}
ind_splitRide = 0
##########################


RideList = np.sort(df_MaschenStart["Ride"].drop_duplicates().to_numpy())

dfMaschen_md = pd.DataFrame()
for ii, ride in enumerate(RideList):
    # df erst nach wagon_ID sortiert, dann jede einzelne wagon_Id nochmals nach der Zeit
    df_Ride = df_MaschenStart[df_MaschenStart['Ride'] == ride]
    df_Ride_sort = df_Ride.sort_values(by=['timestamp_measure_position'])

    ########################## feature moving_direction2
    # Differenz von Geo(i) und Geo(i+1)
    diff_lat = df_Ride_sort['latitude'].diff()
    diff_long = df_Ride_sort['longitude'].diff()
    
    # da der erste Eintrag sonst NaN wäre, wird hier der erste Wert gleich dem zweiten gesetzt
    
    if df_Ride_sort.shape[0]>1: # da iloc[1] sonst nicht existieren würde
        diff_lat.iloc[0] = 0
        diff_long.iloc[0] = 0
        
    sum_lat = diff_lat.sum()
    sum_long = diff_long.sum()
    
    masch_Nord = [53.417188, 10.040325]
    masch_Sud  = [53.397906, 10.070178]
    masch_diff_lat = (masch_Nord[0] - masch_Sud[0])*0.7
    masch_diff_long = (masch_Nord[1] - masch_Sud[1])*0.7
    
    
    if (sum_lat > masch_diff_lat) & (sum_long < -masch_diff_long):
        moving_direction2 = np.array(np.ones(df_Ride_sort.shape[0])*2)
    elif (sum_lat < -masch_diff_lat) & (sum_long > masch_diff_long):
        moving_direction2 = np.array(np.ones(df_Ride_sort.shape[0])*1)
    else:
        moving_direction2 = np.array(np.ones(df_Ride_sort.shape[0])*-1)
        
    # neues Feature zum df hinzufügen
    df_Ride_sort['moving_direction2'] = moving_direction2  

    
    ####################### feature moving_direction 
    
    mean_latMaschen = (max(df_Ride_sort['latitude'])+min(df_Ride_sort['latitude']))/2
    if df_Ride_sort['latitude'].iloc[0] > mean_latMaschen:
        moving_direction = np.array(np.ones(df_Ride_sort.shape[0])*1)
    else: moving_direction = np.array(np.ones(df_Ride_sort.shape[0])*2)
    
    # neues Feature zum df hinzufügen
    df_Ride_sort['moving_direction'] = moving_direction   
    
    ####################### feature moving_direction 3
    moving_direction3 = np.array(np.ones(len(df_Ride_sort))*-1)
    
    # https://www.quora.com/How-many-meters-make-up-a-degree-of-longitude-latitude-on-Earth
    # each degree of latitude is about 111.111 km or 111,111 metres.
    # As we move away from the equator, the lines become closer - the distance the becomes 111,111*cos(latitude)
   
    if GeoFilter == 'lat and long with sensor inaccuracy':
        # hier werden die einzelnen Zustände definiert (-1/0/1/2)
        moving_direction3[(diff_lat>1/111111) & (diff_long<-1/111111)] = 2
        moving_direction3[(diff_lat<-1/111111) & (diff_long>1/111111)] = 1
        moving_direction3[(diff_lat>=-1/111111) & (diff_lat<=1/111111) & (diff_long<=1/111111)&(diff_long>=-1/111111)] = 0
    elif GeoFilter == 'lat and long without sensor inaccuracy':  
        # Alternative, ohne Ungenauigkeit der Sensoren zu berücksichtigen...
        moving_direction3[(diff_lat>0) & (diff_long<0)] = 2
        moving_direction3[(diff_lat<0) & (diff_long>0)] = 1
        moving_direction3[(diff_lat==0) & (diff_long==0)] = 0
    elif GeoFilter == 'only lat up and down': 
        # Alternative, in der nur oben und unten sortiert wird...
        moving_direction3[(diff_lat>1/111111)] = 2
        moving_direction3[(diff_lat<-1/111111)] = 1
        moving_direction3[(diff_lat>=-1/111111) & (diff_lat<=1/111111)] = 0
    else:
        assert 0, "please select a Filtering Method for GeoFilter: 'lat and long with sensor inaccuracy' or 'lat and long without sensor inaccuracy' or 'only lat up and down'"

    df_Ride_sort['moving_direction3'] = moving_direction3  
    ####################### feature movement_count 
    
    moveCountWagon = []          
    pos_movState_0 = np.where(df_Ride_sort['movement_state'] == 0)[0] # überall, wo der Wagon parkt und nur alle Einträge während einer Fahrt
    diff_pos_movState_0 = np.diff(pos_movState_0) # die Differenz der Positionen gibt auskunft darüber, ob mehrere Einträge hintereinander stehen beschreiben

    pos_new_mov  = np.delete(pos_movState_0, (np.where(diff_pos_movState_0 == 1)[0]-1), axis=0) # remove rows where the wagon is parking for a longer period
    pos_new_mov += 1
    pos_new_mov = np.append(pd.Series(data=[0]),pos_new_mov,  axis=0) # at first position starts the first count
    
    if pos_new_mov[-1] != len(df_Ride_sort):
        pos_new_mov = np.append(pos_new_mov,pd.Series(data=[len(df_Ride_sort)]),  axis=0)
    
    movement_count = np.array(np.ones(len(df_Ride_sort)))
    count = 0

    for ii, pos in enumerate(pos_new_mov):

        if ii < len(pos_new_mov)-1: # because otherwise ii+1 would cause an error
            movement_count[pos:pos_new_mov[ii+1]] = count
            count += 1
        else: movement_count[pos:] = count
                  
    moveCountWagon = np.append(moveCountWagon,movement_count)

    # neues Feature zum df hinzufügen
    df_Ride_sort['movement_count'] = moveCountWagon
    
    #######################
    
    # wieder zu einem großen df zusammenfügen
    dfMaschen_md = dfMaschen_md.append(df_Ride_sort)
    
    
# einen Überblick erhalten, wie viele Geo-Einträge wirklich diagonal sind
print('Anzahl der einzelnen Zustände von moving_direction2: \n'+str(dfMaschen_md['moving_direction2'].value_counts()))   
print('Anzahl der einzelnen Zustände: \n'+str(dfMaschen_md['movement_count'].value_counts()))     
# alle nicht 0/1/2 zugeordneten Daten werden entfernt    
#dfMaschen_md = dfMaschen_md[dfMaschen_md['moving_direction2']>=0]



### weiteres Feature: speed_group

|Nummer|Bedeutung|
|:-:|:-|
|3|Geschwindigkeit größer als 30 km/h|
|2|Geschwindigkeit zwischen 5 km/h und 30 km/h|
|1|Geschwindigkeit zwischen als 0 km/h und 5 km/h |
|0|Geschwindigkeit genau 0 km/h|

In [ ]:
indSpeed3 = np.where((dfMaschen_md['GNSS_velocity']>15))
indSpeed2 = np.where((dfMaschen_md['GNSS_velocity']>=5) & (dfMaschen_md['GNSS_velocity']<=15))
indSpeed1 = np.where((dfMaschen_md['GNSS_velocity']>0) & (dfMaschen_md['GNSS_velocity']<5))
indSpeed0 = np.where((dfMaschen_md['GNSS_velocity']==0))

dfMaschen_md['speed_group'] = -1
dfMaschen_md.speed_group.iloc[indSpeed3] = 3
dfMaschen_md.speed_group.iloc[indSpeed2] = 2
dfMaschen_md.speed_group.iloc[indSpeed1] = 1
dfMaschen_md.speed_group.iloc[indSpeed0] = 0

In [ ]:
safePath = os.path.join("..", "..", "data", "DfMaschenPlot.pickle")
dfMaschen_md.to_pickle(safePath)

## Analyse der Ergebnisse des Feature-Engineering

In [ ]:
pathToMaschenFile = os.path.join("..", "..", "data", "DfMaschenPlot.pickle")
dfMaschen_md = pd.read_pickle(pathToMaschenFile)

#### Plots der unterschiedlichen moving_directions

In [ ]:
cFeature= 'moving_direction2' # moving_direction/moving_direction2/moving_direction3
map_maschen = plt.imread(os.path.join('..', '..', 'data', 'map.png'))
BBox = (10.0320, 10.0826, 53.3925, 53.4228)

# 2D-Plot, der Maschen von oben
X = dfMaschen_md[dfMaschen_md[cFeature]==2]

fig, ax = plt.subplots(ncols=2, figsize = (7,4.666), constrained_layout=True, sharey=True)
ax[0].set_title('Zugfahrt von Nord nach Süd')
ax[0].set_ylabel('Latitude (Längengrad)', fontsize = 11)
ax[0].set_xlabel('Longitude (Breitengrad)', fontsize = 11)
ax[0].scatter( Y['longitude'],Y['latitude'], s = 1, zorder=1, c='black', marker='.', alpha=0.2)

Y = dfMaschen_md[dfMaschen_md[cFeature]==1]

ax[1].set_title('Zugfahrt von Süd nach Nord')
ax[1].set_xlabel('Longitude (Breitengrad)', fontsize = 11)
ax[1].scatter( X['longitude'],X['latitude'], s = 1, zorder=1, c='black', marker='.', alpha=0.2)

for axes in ax:
    axes.set_xlim(BBox[0], BBox[1])
    axes.set_ylim(BBox[2], BBox[3])
    axes.imshow(map_maschen, extent= BBox, zorder=0, aspect='equal')
    axes.set_aspect(1.5)
    axes.grid(False)

fig.savefig(os.path.join('..', 'doc', 'north_south_maschen.png'))

#### Plot von speed_group

In [ ]:
cFeature= 'speed_group' # moving_direction2

# 2D-Plot, der Maschen von oben
fig = plt.figure(figsize = (20,7))

ZZ = dfMaschen_md[dfMaschen_md[cFeature]==0]

ax = fig.add_subplot(1,4,1)
ax.set_title('Zugfahrt mit Geschw. größer 30 km/h')
ax.set_xlabel('longitude', fontsize = 15)
#ax.set_ylabel('latitude', fontsize = 15)
ax.scatter( ZZ['longitude'],ZZ['latitude'],c =ZZ[cFeature], s = 0.01)

X = dfMaschen_md[dfMaschen_md[cFeature]==1]

ax = fig.add_subplot(1,4,2)
ax.set_title('Zugfahrt mit Geschw. kleiner 5 km/h')
ax.set_xlabel('longitude', fontsize = 15)
ax.set_ylabel('latitude', fontsize = 15)
ax.scatter( X['longitude'],X['latitude'],c =X[cFeature], s = 0.01)

Y = dfMaschen_md[dfMaschen_md[cFeature]==2]

ax = fig.add_subplot(1,4,3)
ax.set_title('Zugfahrt mit Geschw. grö gl 5 km/h und klei gl 30 km/h')
ax.set_xlabel('longitude', fontsize = 15)
#ax.set_ylabel('latitude', fontsize = 15)
ax.scatter( Y['longitude'],Y['latitude'],c =Y[cFeature], s = 0.01)


Z = dfMaschen_md[dfMaschen_md[cFeature]==3]

ax = fig.add_subplot(1,4,4)
ax.set_title('Zugfahrt mit Geschw. größer 30 km/h')
ax.set_xlabel('longitude', fontsize = 15)
#ax.set_ylabel('latitude', fontsize = 15)
ax.scatter( Z['longitude'],Z['latitude'],c =Z[cFeature], s = 0.01)



In [ ]:
# Analyse, ob timestamp_measure_movement_state und timestamp_measure_position übereinstimmen

str1 ='Die minimale Differenz zwischen timestamp_measure_position und timestamp_measure_movement_state beträgt: '
minTimeDiff = np.min(dfMaschen_md[dfMaschen_md['timestamp_measure_movement_state']!='NaT']['timestamp_measure_position']-dfMaschen_md[dfMaschen_md['timestamp_measure_movement_state']!='NaT']['timestamp_measure_movement_state'])
print(str1+str(minTimeDiff)+' Sekunden')

str2 ='Die durchschnittliche Differenz zwischen timestamp_measure_position und timestamp_measure_movement_state beträgt: '
meanTimeDiff = np.mean(dfMaschen_md[dfMaschen_md['timestamp_measure_movement_state']!='NaT']['timestamp_measure_position']-dfMaschen_md[dfMaschen_md['timestamp_measure_movement_state']!='NaT']['timestamp_measure_movement_state'])
meanStdDiff = np.std(dfMaschen_md[dfMaschen_md['timestamp_measure_movement_state']!='NaT']['timestamp_measure_position']-dfMaschen_md[dfMaschen_md['timestamp_measure_movement_state']!='NaT']['timestamp_measure_movement_state'])
print(str2+str(meanTimeDiff)+'+-'+str(meanStdDiff)+'Sekunden')

str3 ='Die maximale Differenz zwischen timestamp_measure_position und timestamp_measure_movement_state beträgt: '
maxTimeDiff = np.max(dfMaschen_md[dfMaschen_md['timestamp_measure_movement_state']!='NaT']['timestamp_measure_position']-dfMaschen_md[dfMaschen_md['timestamp_measure_movement_state']!='NaT']['timestamp_measure_movement_state'])
print(str3+str(maxTimeDiff)+' Sekunden')

# Ergebnis: Ja, da sie maximal 3 Minuten auseinander liegen

#### Analyse einer Fahrt durch Maschen

In [ ]:
# RideList 3335, 3765, 1619, 4839, 3587, 349, 4000, 5168, mit 0.5: 3836, 4925 mit 20min: 7490 4529

# Überblick durch Geodaten und the wagon moving state"
RideList = np.sort(dfMaschen_md["Ride"].drop_duplicates().to_numpy())

from random import randrange
radn_ind = randrange(len(RideList))

if 1:
    Ride2Plot = RideList[radn_ind]
else: Ride2Plot = 35489

X = dfMaschen_md[dfMaschen_md["Ride"] == Ride2Plot]

map_maschen = plt.imread(os.path.join('..', '..', 'data', 'map.png'))
BBox = (10.0208, 10.1031, 53.3867, 53.4303)


X_normal = dfMaschen_md[dfMaschen_md["Ride"] == 14465]

fig, ax = plt.subplots(ncols=2, figsize = (7,4.666), constrained_layout=True, sharey=True)
ax[0].plot(X_normal['longitude'],
           X_normal['latitude'], color='black', zorder=1, linewidth=0.5)
ax[0].scatter(X_normal['longitude'],
           X_normal['latitude'], s = 6, c='black', zorder=2, marker='.')

X_fail_standing = dfMaschen_md[dfMaschen_md["Ride"] == 35489]
ax[1].plot(X_fail_standing['longitude'],
           X_fail_standing['latitude'], color='red', zorder=1, linewidth=0.5)
ax[1].scatter(X_fail_standing['longitude'],
           X_fail_standing['latitude'], s = 6, c='red', zorder=2, marker='.')


X_jumping = dfMaschen_md[dfMaschen_md["Ride"] == 1043]
ax[1].plot(X_jumping['longitude'],
           X_jumping['latitude'], color='blue', zorder=1, linewidth=0.5)
ax[1].scatter(X_jumping['longitude'],
           X_jumping['latitude'], s = 6, c='blue', zorder=2, marker='.')




X_middle_start = dfMaschen_md[dfMaschen_md["Ride"] == 2842]
ax[1].plot(X_middle_start['longitude'],
           X_middle_start['latitude'], color='orange', zorder=1, linewidth=0.5)
ax[1].scatter(X_middle_start['longitude'],
           X_middle_start['latitude'], s = 6, c='orange', zorder=2, marker='.')

for axes in ax:
    axes.set_xlim(BBox[0], BBox[1])
    axes.set_ylim(BBox[2], BBox[3])
    axes.imshow(map_maschen, extent= BBox, zorder=0, aspect='equal')
    axes.set_aspect(1.5)
    axes.grid(False)

ax[0].set_xlabel('Longitude (Breitengrad)', fontsize = 15)
ax[1].set_xlabel('Longitude (Breitengrad)', fontsize = 15)

ax[0].set_ylabel('Latitude (Lägengrad)', fontsize = 15)

fig.savefig(os.path.join('..', 'doc', 'single_rides.pdf'))


inDHMS = list(np.ones(len(X)))

# timestamp Sekunden in Tage Stunden etc umrechnen 
import datetime 
  
def convert(n): 
    return str(datetime.timedelta(seconds = n))



for ii, time in enumerate(X['timestamp_measure_position'].values):
    inDHMS[ii] = convert(time)

X.loc[:,'time_position'] = inDHMS
    
print(X[['wagon_ID','movement_state','time_position' ]].head(30))

#### Analyse der Datenpunkte eines Wagons in Maschen

In [ ]:
wagonIDList = np.sort(dfMaschen_md["wagon_ID"].drop_duplicates().to_numpy())


X_ride1 = dfMaschen_md[dfMaschen_md["Ride"] == 36636]
X_ride2 = dfMaschen_md[dfMaschen_md["Ride"] == 36637]


Y = dfMaschen_md[dfMaschen_md["wagon_ID"] == 1176603780144674]
X = Y.sort_values(by=['timestamp_measure_position'])


fig, ax = plt.subplots(ncols=2, figsize = (7,4.666), constrained_layout=True, sharey=True)
ax[0].plot(X['longitude'],
           X['latitude'], color='black', zorder=1, linewidth=0.5)
ax[0].scatter(X_ride1['longitude'],
           X_ride1['latitude'], s = 6, c='darkorchid', zorder=2, marker='.')
ax[0].scatter(X_ride2['longitude'],
           X_ride2['latitude'], s = 6, c='midnightblue', zorder=2, marker='.')

ax[1].plot(X_ride1['longitude'],
           X_ride1['latitude'], color='darkorchid', zorder=1, linewidth=0.5)
ax[1].plot(X_ride2['longitude'],
           X_ride2['latitude'], color='midnightblue', zorder=1, linewidth=0.5)
ax[1].scatter(X_ride1['longitude'],
           X_ride1['latitude'], s = 6, c='darkorchid', zorder=2, marker='.')
ax[1].scatter(X_ride2['longitude'],
           X_ride2['latitude'], s = 6, c='midnightblue', zorder=2, marker='.')



for axes in ax:
    axes.set_xlim(BBox[0], BBox[1])
    axes.set_ylim(BBox[2], BBox[3])
    axes.imshow(map_maschen, extent= BBox, zorder=0, aspect='equal')
    axes.set_aspect(1.5)
    axes.grid(False)

ax[0].set_xlabel('Longitude (Breitengrad)', fontsize = 15)
ax[1].set_xlabel('Longitude (Breitengrad)', fontsize = 15)

ax[0].set_ylabel('Latitude (Lägengrad)', fontsize = 15)
fig.savefig(os.path.join('..', 'doc', 'wagonID.pdf'))


In [ ]:
Ride2Plot
#35489 --> Ride Standing
#3956 --> Fährt rein und dannn wieder zurück
#1043 --> Ride der hin und her springt
#2842 --> Ride beginnt mitten drin und hört auch wieder auf

#9990 --> ist okay gut
#14165
#14465

#### Analyse des Features 'movement_count': 

##### Graphische Analyse von movement_count

In [ ]:
# Überblick durch Geodaten und the wagon moving state"AnzahlCount
AnzahlCount = 3

X = dfMaschen_md[dfMaschen_md['movement_count']<5]

plt.figure(figsize=(10, 10)) 
ax = plt.axes(projection='3d')
ax.set_xlabel('longitude', fontsize = 15)
ax.set_ylabel('latitude', fontsize = 15)
ax.set_zlabel('moving_direction', fontsize = 15)
ax.set_title('Analyse der movement_count \n mit Berücksichtigung der moving_direction' , fontsize = 20)
ax.scatter3D(X['longitude'],
           X['latitude'],
           X['moving_direction'],
           c = X['movement_count'], s = 0.5)
ax.legend( ['die ersten '+str(AnzahlCount)+' moving_count farblich'])
plt.show()

fig = plt.figure(figsize = (10,5))
fig.suptitle('Analyse der movement_count \n mit Berücksichtigung der moving_direction' , fontsize = 20)
ax = fig.add_subplot(1,2,1) 
ax.set_xlabel('longitude', fontsize = 15)
ax.set_ylabel('latitude', fontsize = 15)
#ax.set_title('3 component PCA', fontsize = 20)
ax.scatter( X['longitude'][X['moving_direction']==1],X['latitude'][X['moving_direction']==1],c=X[X['moving_direction']==1]["movement_count"], s = 0.3)
ax.grid()

ax = fig.add_subplot(1,2,2) 
ax.set_xlabel('longitude', fontsize = 15)
#ax.set_ylabel('Principal Component 3', fontsize = 15)
#ax.set_title('3 component PCA', fontsize = 20)
ax.scatter( X['longitude'][X['moving_direction']==2],X['latitude'][X['moving_direction']==2],c=X[X['moving_direction']==2]["movement_count"], s = 0.3)
ax.grid()

##### Verteilung der movment_count

In [ ]:
from collections import Counter

count_movCont = Counter(dfMaschen_md['movement_count'])

# Überblick durch Geodaten und the wagon moving state"
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(7,7))
ax.set_xlabel('movement_count', fontsize = 15)
ax.set_ylabel('Anzahl der Wagoneinträge mit movement_count', fontsize = 15)
ax.set_title('Analyse der movement_count-Verteilung', fontsize = 20)
ax.bar(count_movCont.keys(),
           count_movCont.values()
           )
plt.show()

## Clustering des Features 'moving_direction'

### Analyse mit DBSCAN: 

#### Standardize the df to work with DBSCAN

In [ ]:
dfMaschen_hard = dfMaschen_md[dfMaschen_md['moving_direction2']>0]


from sklearn.preprocessing import StandardScaler
features5 = ['latitude','longitude','moving_direction2'] # Separating out the features
standMasch_df5 = dfMaschen_hard.loc[:, features5].values # Separating out the target
standMasch_df5 = StandardScaler().fit_transform(standMasch_df5)

#### DBSCAN directly with standardized df maschen

In [ ]:
# DBSCAN tries to cluster the data
from sklearn.cluster import DBSCAN
db = DBSCAN(eps=0.045, min_samples=200).fit(standMasch_df5)
labeldb5 =  db.labels_

# Um zu sehen, wie viele Cluster erkannt wurden
print('Die folgenden Cluster wurden erkannt: '+str(np.unique(labeldb5)))
      
# hier werden die PlotDaten hergerichtet und erweitert um die Labels      
X = dfMaschen_hard[['latitude','longitude','moving_direction2']] #[dfMaschen_md['moving_direction']==2]
X["label"] = labeldb5
      
# da Cluster [-1] und [0] die Übersicht verringern, können sie hier hiermit ausgeblendet werden      
Y = X[X["label"]>-2]


# 3D-Plot, um zu erkennen, von welcher Ebene welche Cluster kommen
fig = plt.figure(figsize = (8,8))
ax = plt.axes(projection='3d')
ax.set_xlabel('longitude')
ax.set_ylabel('latitude')
ax.set_zlabel('moving_direction2')
ax.set_title('Moving_direction as new feature')
ax.scatter3D(Y['longitude'], Y['latitude'],Y['moving_direction2'],c = Y["label"], s = 0.05)
ax.grid()

Y_clusters = Y[Y['label']>=0]
Y_no_clusters = Y[Y['label']<0]

# 2D-Plot, der Maschen von oben zeigt mit Clustern
fig, ax = plt.subplots(ncols=2, figsize = (7,4.666), constrained_layout=True, sharey=True)
ax[0].scatter(Y_no_clusters['longitude'][Y_no_clusters['moving_direction2']== 1],Y_no_clusters['latitude'][Y_no_clusters['moving_direction2']== 1],c = 'dimgray', s = 1, marker='.', alpha=0.5)
ax[1].scatter(Y_no_clusters['longitude'][Y_no_clusters['moving_direction2']== 2],Y_no_clusters['latitude'][Y_no_clusters['moving_direction2']== 2],c = 'dimgray', s = 1, marker='.', alpha=0.5)

ax[0].scatter(Y_clusters['longitude'][Y_clusters['moving_direction2']== 1],Y_clusters['latitude'][Y_clusters['moving_direction2']== 1],c = Y_clusters[Y_clusters['moving_direction2']== 1]["label"], s = 1, cmap='Paired', marker='x', alpha=0.5)
ax[1].scatter(Y_clusters['longitude'][Y_clusters['moving_direction2']== 2],Y_clusters['latitude'][Y_clusters['moving_direction2']== 2],c = Y_clusters[Y_clusters['moving_direction2']== 2]["label"], s = 1, cmap='Paired', marker='x', alpha=0.5)


for axes in ax:
    axes.set_xlim(BBox[0], BBox[1])
    axes.set_ylim(BBox[2], BBox[3])
    axes.imshow(map_maschen, extent= BBox, zorder=0, aspect='equal')
    axes.set_aspect(1.5)
    axes.grid(False)

ax[0].set_title('Zugfahrt von Nord nach Süd')
ax[1].set_title('Zugfahrt von Süd nach Nord')

ax[0].set_xlabel('Longitude (Breitengrad)')
ax[1].set_xlabel('Longitude (Breitengrad)')
ax[0].set_ylabel('Latitude (Lägengrad)')

fig.savefig(os.path.join('..', 'doc', 'DBSCAN.png'))

### (verworfen) Analyse der 'moving_direction' mit PCA und danach DBSCAN:

#### Standardize the df to work with DBSCAN

In [ ]:
from sklearn.preprocessing import StandardScaler
features5 = ['latitude','longitude','moving_direction']# Separating out the features
standMasch_df5 = dfMaschen_md.loc[:, features5].values# Separating out the target
standMasch_df5 = StandardScaler().fit_transform(standMasch_df5)

#### PCA of Standardized df Maschen

In [ ]:
# die bisher direkt geclusterten Daten werden nun zuerst anhand von PCA neu geordnet, um eventuell bessere Clusterergebnisse zu erhalten
from sklearn.decomposition import PCA
pca_maschen5 = PCA(n_components=3)
principalComponents5 = pca_maschen5.fit_transform(standMasch_df5)
principalDf5 = pd.DataFrame(data = principalComponents5, columns = ['principal component 1', 'principal component 2','principal component 3']) 

##### Plotten des PCA-Ergebnisses

In [ ]:
fig = plt.figure(figsize = (5,5))
ax = plt.axes(projection='3d')
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_zlabel('Principal Component 3', fontsize = 15)
ax.set_title('3 component PCA', fontsize = 20)
ax.scatter3D(principalDf5['principal component 1'],principalDf5['principal component 2'],principalDf5['principal component 3'], s=0.3)
ax.grid()

In [ ]:
fig = plt.figure(figsize = (5,5))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 2', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('PCA von DF mit Moving_direction', fontsize = 20)
ax.scatter(principalDf5['principal component 1'],principalDf5['principal component 3'], s =0.3)
ax.grid()

#### DBSCAN von PCA mit moving_direction

In [ ]:
from sklearn.cluster import DBSCAN
db = DBSCAN(eps=0.045, min_samples=200).fit(principalDf5)
labeldb5 =  db.labels_

print('Anzahl an Clustern: '+ str(np.unique(labeldb5)))

# hier werden die PlotDaten hergerichtet und erweitert um die Labels      
X = principalDf5
X["label"] = labeldb5
      
# da Cluster [-1] und [0] die Übersicht verringern, können sie hier hiermit ausgeblendet werden      
Y = X[X["label"]>-2]

fig = plt.figure(figsize = (8,8))
ax = plt.axes(projection='3d')
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_zlabel('Principal Component 3', fontsize = 15)
ax.set_title('3 component PCA', fontsize = 20)

ax.scatter3D(Y['principal component 1'],Y['principal component 2'],Y['principal component 3'],c = Y["label"], s = 3)
ax.grid()


fig = plt.figure(figsize = (10,5))
ax = fig.add_subplot(1,2,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 3', fontsize = 15)
ax.set_title('3 component PCA', fontsize = 20)
ax.scatter( Y['principal component 1'][Y['principal component 2']< 0],Y['principal component 3'][Y['principal component 2']< 0],c = Y[Y['principal component 2']< 0]["label"], s = 0.1)
ax.grid()

ax = fig.add_subplot(1,2,2) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 3', fontsize = 15)
ax.set_title('3 component PCA', fontsize = 20)
ax.scatter( Y['principal component 1'][Y['principal component 2']> 0],Y['principal component 3'][Y['principal component 2']> 0],c = Y[Y['principal component 2']> 0]["label"], s = 0.1)
ax.grid()